In [1]:
import pandas as pd

## Read Station / Weather Data

In [2]:
# Note: Without latin-1 encoding, receive error: "UnicodeDecodeError, invalid continuation byte"
df_stations = pd.read_csv('data/location_data_working.csv', encoding='latin-1')
df_stations.head()

,StationID,FIPS,County,StateID,State,Latitude,Longitude
0,US009052008,46099,Minnehaha County,SD,South Dakota,43.7333,-96.6333
1,US10RMHS145,08069,Larimer County,CO,Colorado,40.5268,-105.1113
2,US10adam001,31001,Adams County,NE,Nebraska,40.5680,-98.5069
3,US10adam002,31001,Adams County,NE,Nebraska,40.5093,-98.5493
4,US10adam003,31001,Adams County,NE,Nebraska,40.4663,-98.6537


In [3]:
def read_normals(filename):
    df = pd.DataFrame(columns=['StationID', 'Jan', 'Feb', 'March', 'April', 'May', 'June', 'July', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec'])
    
    # Simply for progress tracking.
    i = 0
    num_lines = sum(1 for _ in open(filename))
    
    with open(filename) as f:
        for line in f:
            # Display progress.
            print('\r', 'Progress: {}%'.format(str(round(i / (num_lines-1) * 100, 3))), end='')
            
            # Uneven spacing, so use split to place each column as item in a `columns` list.
            columns = line.split()
            
            # Append current row to dataframe.
            #   Note: Only care about the US stations.
            if columns[0][0:2] == 'US':
                df.loc[len(df)] = columns
            
            # Simply for progress tracking.
            i += 1

    return df

In [4]:
df_tavg = read_normals('data/mly-tavg-normal.txt')
df_tavg.head()

 Progress: 100.0%%

,StationID,Jan,Feb,March,April,May,June,July,Aug,Sept,Oct,Nov,Dec
0,USC00010063,397Q,437Q,515Q,596Q,678Q,749Q,784Q,783Q,718Q,607Q,515Q,423Q
1,USC00010160,434S,470S,544S,613S,696S,767S,801S,792S,734S,630S,540S,456S
2,USC00010178,434R,472R,553R,627R,705R,781R,809R,805R,747R,637R,543R,458R
3,USC00010252,466S,502S,565S,625S,705S,773S,796S,793S,747S,648S,560S,488R
4,USC00010260,377Q,413Q,501Q,588Q,671Q,747Q,785Q,778Q,712Q,600Q,500Q,404Q


In [5]:
df_prcp = read_normals('data/mly-prcp-normal.txt')
df_prcp.head()

 Progress: 100.0%%

,StationID,Jan,Feb,March,April,May,June,July,Aug,Sept,Oct,Nov,Dec
0,USC00010008,530S,507S,580S,377S,382S,520C,594S,463S,353S,311S,461S,473S
1,USC00010063,503R,529R,525R,478R,548R,483R,483R,375R,427R,377R,505R,591R
2,USC00010140,501C,530S,564C,412S,392S,436S,495S,422S,347C,341C,493C,498S
3,USC00010160,521S,535S,549S,411C,433C,445C,531S,450S,410S,308S,479S,490S
4,USC00010178,524R,559S,527R,486R,460R,488R,499R,357R,376R,397R,484R,460R


In [6]:
df_snow = read_normals('data/mly-snow-normal.txt')
df_snow.head()

 Progress: 100.0%%

,StationID,Jan,Feb,March,April,May,June,July,Aug,Sept,Oct,Nov,Dec
0,USC00010008,0S,0S,0S,0S,0S,0C,0S,0S,0S,0S,0S,1P
1,USC00010063,4P,-7777P,0P,0P,0P,0P,0P,0P,0P,0P,0P,0P
2,USC00010140,-7777S,0S,4S,0S,0C,0C,0S,0S,0C,0C,0C,0S
3,USC00010160,3S,0S,-7777S,-7777S,0C,0C,0S,0S,0S,0S,0S,1S
4,USC00010184,0P,0S,0S,0S,0S,0C,0S,0S,0C,0C,0C,0S


## Clean Weather Data
- Delete flags (e.g., 39.7Q => 39.7). Note: Flags simply indicate the completeness of the data.
- Format values into appropriate form (e.g., 397Q => 39.7Q). Note: tavg => tenths, prcp => hundredths, snow => tenths.
- Convert special values (e.g., -9999, -8888, -7777, -6666, -5555). Reference: https://www.ncei.noaa.gov/pub/data/normals/1981-2010/readme.txt

In [7]:
def remove_flag(val):
    # Note: Flag will be the last character.
    return val[:-1]


def convert_special_value(val):
    # Convert special value, if present.
    #   "-9999", "-8888", "-6666", "-5555" represent undefined/unavailable data, so convert to "null".
    #   "-7777" represents "a non-zero value that would round to zero, for variables bound by zero", but
    #   simply round to 0 for our purposes.
    special_values = ['-9999', '-8888', '-6666', '-5555']
    if val in special_values:
        return 'null'
    elif val == '-7777':
        # Note: `insert_decimal(...)` will format this zero appropriately.
        return '0'
    
    return val


def insert_decimal(val, decimals):
    # Insert decimal place.
    #   Note: Does not account for 3 or more decimals.
    if decimals > 0:
        # Account for negative value.
        is_negative = False
        if val[0] == '-':
            is_negative = True
            val = val[1:]
        
        ret = None
        if decimals == 1:
            if len(val) == 1:
                # Insert decimal before.
                ret = '0.' + val
            else:
                # Insert decimal before last digit.
                ret = val[:len(val)-1] + '.' + val[len(val)-1]
        elif decimals == 2:
            if len(val) == 2:
                # Insert decimal before.
                ret = '0.' + val
            elif len(val) == 1:
                # Insert decimal and zero.
                ret = '0.0' + val
            else:
                ret = val[:len(val)-2] + '.' + val[len(val)-2:]

        # Account for negative value.
        ret = float(ret)
        if is_negative:
            ret *= -1
            
        return ret
    else:
        # No decimals, so simply return value as an integer.
        return int(val)
        

def clean_data(val, decimals):
    val = remove_flag(val)

    val = convert_special_value(val)
    if val == 'null':
        return val
    
    val = insert_decimal(val, decimals)
    
    return val


def clean_df(df, decimals):
    station_ids = df.loc[:, 'StationID']
    df = df.loc[:, df.columns != 'StationID'].apply(lambda row: row.apply(lambda col: clean_data(col, decimals)))
    df.insert(0, 'StationID', station_ids)
    return df

In [8]:
df_tavg = clean_df(df_tavg, 1)
df_tavg.head()

,StationID,Jan,Feb,March,April,May,June,July,Aug,Sept,Oct,Nov,Dec
0,USC00010063,39.7,43.7,51.5,59.6,67.8,74.9,78.4,78.3,71.8,60.7,51.5,42.3
1,USC00010160,43.4,47.0,54.4,61.3,69.6,76.7,80.1,79.2,73.4,63.0,54.0,45.6
2,USC00010178,43.4,47.2,55.3,62.7,70.5,78.1,80.9,80.5,74.7,63.7,54.3,45.8
3,USC00010252,46.6,50.2,56.5,62.5,70.5,77.3,79.6,79.3,74.7,64.8,56.0,48.8
4,USC00010260,37.7,41.3,50.1,58.8,67.1,74.7,78.5,77.8,71.2,60.0,50.0,40.4


In [9]:
df_prcp = clean_df(df_prcp, 2)
df_prcp.head()

,StationID,Jan,Feb,March,April,May,June,July,Aug,Sept,Oct,Nov,Dec
0,USC00010008,5.30,5.07,5.80,3.77,3.82,5.20,5.94,4.63,3.53,3.11,4.61,4.73
1,USC00010063,5.03,5.29,5.25,4.78,5.48,4.83,4.83,3.75,4.27,3.77,5.05,5.91
2,USC00010140,5.01,5.30,5.64,4.12,3.92,4.36,4.95,4.22,3.47,3.41,4.93,4.98
3,USC00010160,5.21,5.35,5.49,4.11,4.33,4.45,5.31,4.50,4.10,3.08,4.79,4.90
4,USC00010178,5.24,5.59,5.27,4.86,4.60,4.88,4.99,3.57,3.76,3.97,4.84,4.60


In [10]:
df_snow = clean_df(df_snow, 1)
df_snow.head()

,StationID,Jan,Feb,March,April,May,June,July,Aug,Sept,Oct,Nov,Dec
0,USC00010008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1
1,USC00010063,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,USC00010140,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,USC00010160,0.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1
4,USC00010184,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Convert StationID to FIPS
Use `location_data.csv` file generated in another script that converted station latitude/longitude to FIPS using an API from FCC: https://geo.fcc.gov/api/census/#!/block/get_block_find

In [11]:
def convert_station_to_fips(df):
    # Combine dataframes, essentially "looking up" StationID's FIPS code in df_stations.
    df = df.join(df_stations.set_index('StationID'), on='StationID')
    # Remove unneeded station data (i.e., StationID, Latitude, Longitude, County, StateID, State).
    #   Note: County, StateID, and State can be extracted from FIPS.
    df = df.drop(columns=['StationID', 'Latitude', 'Longitude', 'County', 'StateID', 'State'])
    return df

In [12]:
df_tavg = convert_station_to_fips(df_tavg)
df_tavg.head()

,Jan,Feb,March,April,May,June,July,Aug,Sept,Oct,Nov,Dec,FIPS
0,39.7,43.7,51.5,59.6,67.8,74.9,78.4,78.3,71.8,60.7,51.5,42.3,01133
1,43.4,47.0,54.4,61.3,69.6,76.7,80.1,79.2,73.4,63.0,54.0,45.6,01123
2,43.4,47.2,55.3,62.7,70.5,78.1,80.9,80.5,74.7,63.7,54.3,45.8,01107
3,46.6,50.2,56.5,62.5,70.5,77.3,79.6,79.3,74.7,64.8,56.0,48.8,01039
4,37.7,41.3,50.1,58.8,67.1,74.7,78.5,77.8,71.2,60.0,50.0,40.4,01077


In [13]:
df_prcp = convert_station_to_fips(df_prcp)
df_prcp.head()

,Jan,Feb,March,April,May,June,July,Aug,Sept,Oct,Nov,Dec,FIPS
0,5.30,5.07,5.80,3.77,3.82,5.20,5.94,4.63,3.53,3.11,4.61,4.73,01067
1,5.03,5.29,5.25,4.78,5.48,4.83,4.83,3.75,4.27,3.77,5.05,5.91,01133
2,5.01,5.30,5.64,4.12,3.92,4.36,4.95,4.22,3.47,3.41,4.93,4.98,01131
3,5.21,5.35,5.49,4.11,4.33,4.45,5.31,4.50,4.10,3.08,4.79,4.90,01123
4,5.24,5.59,5.27,4.86,4.60,4.88,4.99,3.57,3.76,3.97,4.84,4.60,01107


In [14]:
df_snow = convert_station_to_fips(df_snow)
df_snow.head()

,Jan,Feb,March,April,May,June,July,Aug,Sept,Oct,Nov,Dec,FIPS
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,01067
1,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01133
2,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01131
3,0.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,01123
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01107


## Clean Weather Data (w/ FIPS)
Some entries may have duplicate FIPS codes -- combine these, averaging their weather data values. Note: There may be duplicate FIPS if several weather stations reside in the same county, since the FIPS codes were extracted from provided latitude/longitude coordinates from the original dataset.

In [15]:
def avg_fips_dupes(df, decimals):
    df_cleaned = df.groupby('FIPS', as_index=False).mean(numeric_only=True).round(decimals)
    return df_cleaned

In [16]:
df_tavg = avg_fips_dupes(df_tavg, 1)
df_tavg.head()

,FIPS,Jan,Feb,March,April,May,June,July,Aug,Sept,Oct,Nov,Dec
0,01003,49.5,52.8,58.8,65.4,73.1,79.0,81.1,80.8,76.8,67.6,59.0,51.8
1,01005,46.7,50.6,57.3,63.9,72.0,78.6,81.2,80.6,75.6,65.7,56.7,48.4
2,01007,42.7,46.5,53.6,60.7,68.8,76.0,79.0,78.7,73.0,62.3,52.8,44.8
3,01009,41.3,45.0,52.7,60.2,68.5,75.7,79.3,78.9,72.6,61.6,52.3,43.7
4,01011,45.3,48.9,55.4,61.6,69.9,77.0,79.8,79.1,74.0,64.0,55.0,47.3


In [17]:
df_prcp = avg_fips_dupes(df_prcp, 2)
df_prcp.head()

,FIPS,Jan,Feb,March,April,May,June,July,Aug,Sept,Oct,Nov,Dec
0,01001,5.04,4.66,5.47,3.90,3.96,3.81,4.87,4.26,3.68,3.12,4.33,4.46
1,01003,5.75,5.40,5.81,4.60,5.05,6.48,8.11,6.90,5.83,4.05,5.15,4.92
2,01005,4.51,4.53,5.74,3.61,3.26,4.60,5.15,3.57,3.62,2.96,4.10,4.49
3,01007,5.46,5.77,5.34,4.48,4.44,4.67,4.88,4.20,4.54,3.45,5.31,5.03
4,01009,5.29,5.50,4.98,4.36,4.73,4.67,5.47,3.31,3.61,3.50,4.90,5.09


In [18]:
df_snow = avg_fips_dupes(df_snow, 1)
df_snow.head()

,FIPS,Jan,Feb,March,April,May,June,July,Aug,Sept,Oct,Nov,Dec
0,01001,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1
1,01003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1
2,01005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,01007,0.2,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01009,0.6,0.3,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1


## Export
Export each weather dataframe into their own CSV file. This enables other scripts to easily use the cleaned/converted data. Note: Merging these dataframes into one will occur in a different script.

In [19]:
df_tavg.to_csv('output/mly-tavg-normal.csv', index=False)
df_prcp.to_csv('output/mly-prcp-normal.csv', index=False)
df_snow.to_csv('output/mly-snow-normal.csv', index=False)